# (B) Data Cleaning & Creating Final Usable Dataframes

In [1]:
import numpy as np 
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import re
import json

import os

# View all columns of a dataframe
pd.set_option('display.max_columns', None)

In [2]:
os.makedirs('../data', exist_ok=True)  

## 00. HSCP (Health and Social Care Partnership) & HB (Health Board) Data

In [3]:
hscp_df = pd.read_csv("../rawdata/hscp16_hscp19.csv")

hscp_df = hscp_df[['HSCP', 'HSCPName']]

In [4]:
hscp_df.to_csv("../data/hscp.csv", index=None)

## 01. GP Practices Contact Details

In [5]:
gp_practices_contact_details_df = pd.read_csv("../rawdata/practice_contactdetails_oct2024-open-data.csv")

In [6]:
gp_practices_contact_details_df = gp_practices_contact_details_df.merge( hscp_df,
    on="HSCP",       
    how="left"
)

In [7]:
gp_practices_relevant_columns = gp_practices_contact_details_df[['PracticeCode', 'GPPracticeName', 'PracticeListSize', 'Postcode', 
                                                                 'HSCPName', 'AddressLine4']]

gp_practices_relevant_columns = gp_practices_relevant_columns.rename(columns={"HSCPName": "Geography"})

In [8]:
def assign_split_geography(addr):
    if isinstance(addr, str) and addr.strip().lower() in ["clackmannanshire", "sauchie"]:
        return "Clackmannanshire"
    else:
        return "Stirling"

rows_to_split = gp_practices_relevant_columns[
    gp_practices_relevant_columns["Geography"] == "Clackmannanshire and Stirling"
]

# Assign new geography for these rows
rows_to_split = rows_to_split.copy()
rows_to_split["Geography"] = rows_to_split["AddressLine4"].apply(assign_split_geography)

# Replace old combined row(s) with the new split ones
gp_practices_relevant_columns = pd.concat([
    gp_practices_relevant_columns[
        gp_practices_relevant_columns["Geography"] != "Clackmannanshire and Stirling"
    ],
    rows_to_split
], ignore_index=True)

In [9]:
gp_practices_relevant_columns.drop(columns=['AddressLine4'], inplace=True)

In [10]:
gp_practices_relevant_columns.to_csv('../data/gp_practices.csv', index=None)

## 02. GP Contact Details

In [11]:
gp_contact_details_df = pd.read_csv("../rawdata/gp_contactdetails_oct2024-open-data.csv")

gp_relevant_columns = gp_contact_details_df[['GeneralMedicalCouncilNumber', 'PracticeCode', 'GPDesignation', 'Forename',
       'MiddleInitial', 'Surname', 'Sex']]

gp_relevant_columns.to_csv('../data/gps.csv', index=None)

## 03. GP Practices List Sizes

In [12]:
gp_practices_list_sizes_df = pd.read_csv("../rawdata/practice_listsizes_oct2024-open-data.csv")

gp_list_sizes = gp_practices_list_sizes_df[['PracticeCode', 'Sex', 'AllAges', 'Ages0to4', 'Ages5to14', 'Ages15to24', 
                                           'Ages25to44', 'Ages45to64', 'Ages65to74', 'Ages75to84', 'Ages85plus']]

# Melt the dataframe
gp_long = gp_list_sizes.melt(
    id_vars=["PracticeCode", "Sex"],   # Columns to keep
    var_name="AgeGroup",               # New column name for previous column headers
    value_name="Count"                 # New column name for values
)

gp_long.to_csv('../data/gp_list_sizes.csv', index=None)

## 04. Disease Prevalence by GP Practice

In [13]:
disease_prevalence_gp_practice_df = pd.read_excel("../rawdata/diseaseprevalence_practice_total.xlsx", sheet_name="DP_Practice_total")

In [14]:
disease_prevalence_gp_practice_df['Age'].unique()

array(['00-04', '05-09', '10-14', '15-19', '20-24', '25-29', '30-34',
       '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69',
       '70-74', '75-79', '80-84', '85plus', 'All'], dtype=object)

`Age` column is divided into 19 age ranges. However, the GP list sizes dataframe has only 8 ranges (0-4, 5-14, 15-24, 25-44, 45-64, 65-74, 75-84, 85+). So we need to create another column and classify these into these 8 categories.

In [15]:
age_mapping = {
    'All': 'AllAges',
    '00-04': 'Ages0to4',
    '05-09': 'Ages5to14', '10-14': 'Ages5to14',
    '15-19': 'Ages15to24', '20-24': 'Ages15to24',
    '25-29': 'Ages25to44', '30-34': 'Ages25to44', '35-39': 'Ages25to44', '40-44': 'Ages25to44',
    '45-49': 'Ages45to64', '50-54': 'Ages45to64', '55-59': 'Ages45to64', '60-64': 'Ages45to64',
    '65-69': 'Ages65to74', '70-74': 'Ages65to74',
    '75-79': 'Ages75to84', '80-84': 'Ages75to84',
    '85plus': 'Ages85plus'
}

In [16]:
disease_prevalence_gp_practice_df['Age_Group'] = (
    disease_prevalence_gp_practice_df['Age'].map(age_mapping)
)

In [17]:
disease_prevalence_relevant_columns = disease_prevalence_gp_practice_df[
    ['PracticeCode', 'Year', 'Age', 'Age_Group', 
       'PatientCount_Asthma', 'PatientCount_Atrial Fibrillation',
       'PatientCount_Cancer', 'PatientCount_Chronic Kidney Disease (CKD)',
       'PatientCount_Chronic Obstructive Pulmonary Disease (COPD)',
       'PatientCount_Coronary Heart Disease (CHD)', 'PatientCount_Dementia',
       'PatientCount_Depression', 'PatientCount_Diabetes',
       'PatientCount_Eating Disorder', 'PatientCount_Epilepsy',
       'PatientCount_Heart Failure', 'PatientCount_Hypertension',
       'PatientCount_Mental Health', 'PatientCount_Osteoporosis',
       'PatientCount_Palliative Care',
       'PatientCount_Peripheral Arterial Disease (PAD)',
       'PatientCount_Rheumatoid Arthritis', 'PatientCount_Stroke and TIA',
       'Change_Asthma', 'Change_Atrial Fibrillation', 'Change_Cancer',
       'Change_Chronic Kidney Disease (CKD)',
       'Change_Chronic Obstructive Pulmonary Disease (COPD)',
       'Change_Coronary Heart Disease (CHD)', 'Change_Dementia',
       'Change_Depression', 'Change_Diabetes', 'Change_Eating Disorder',
       'Change_Epilepsy', 'Change_Heart Failure', 'Change_Hypertension',
       'Change_Mental Health', 'Change_Osteoporosis', 'Change_Palliative Care',
       'Change_Peripheral Arterial Disease (PAD)',
       'Change_Rheumatoid Arthritis', 'Change_Stroke and TIA', 'Rate_Asthma',
       'Rate_Atrial Fibrillation', 'Rate_Cancer',
       'Rate_Chronic Kidney Disease (CKD)',
       'Rate_Chronic Obstructive Pulmonary Disease (COPD)',
       'Rate_Coronary Heart Disease (CHD)', 'Rate_Dementia', 'Rate_Depression',
       'Rate_Diabetes', 'Rate_Eating Disorder', 'Rate_Epilepsy',
       'Rate_Heart Failure', 'Rate_Hypertension', 'Rate_Mental Health',
       'Rate_Osteoporosis', 'Rate_Palliative Care',
       'Rate_Peripheral Arterial Disease (PAD)', 'Rate_Rheumatoid Arthritis',
       'Rate_Stroke and TIA']
]

In [18]:
df = disease_prevalence_relevant_columns.copy()

diseases = sorted(
    set(col.replace("PatientCount_", "")
             .replace("Change_", "")
             .replace("Rate_", "")
        for col in df.columns if col.startswith(("PatientCount_", "Change_", "Rate_")))
)

for disease in diseases:
    patient_col = f"PatientCount_{disease}"
    change_col = f"Change_{disease}"
    rate_col = f"Rate_{disease}"
    
    df[disease] = df.apply(lambda row: [row[patient_col], row[change_col], row[rate_col]], axis=1)

drop_cols = [f"PatientCount_{d}" for d in diseases] + \
            [f"Change_{d}" for d in diseases] + \
            [f"Rate_{d}" for d in diseases]

df = df.drop(columns=drop_cols)

meta_cols = ["PracticeCode", "Year", "Age", "Age_Group"]  
disease_cols = [c for c in df.columns if c not in meta_cols]

# Melt to long format
df_melted = df.melt(
    id_vars=meta_cols,
    value_vars=disease_cols,
    var_name="Disease",
    value_name="Values"
)

df_split = pd.DataFrame(df_melted["Values"].tolist(), 
                        columns=["PatientCount", "Change", "Rate"])

df_melted = pd.concat([df_melted.drop(columns=["Values"]), df_split], axis=1)

df_melted.to_csv('../data/disease_prevalence.csv', index=None)

## 05. GP Activity by HB

In [19]:
gp_activity_hb_df = pd.read_excel("../rawdata/data-may2025.xlsx", sheet_name="tab6final")
gp_activity_hb_df.drop(columns=['DNA'], inplace=True)

In [20]:
gp_activity_hb_df.to_csv('../data/gp_activity.csv', index=None)

## 06. Hospital Activity Data

In [21]:
def process_hospital_activity_df(df, lookup_col='lookup'):
    # Step 1: Extract Year
    df['Year'] = df[lookup_col].str[:7]
    
    # Step 2: Remove first 7 characters
    df[lookup_col] = df[lookup_col].str[7:]
    
    # Step 3: Extract Geography (with special case)
    if df[lookup_col].str.startswith('p').all():
        df[lookup_col] = df[lookup_col].str[1:]
    
    special_case = "All Scottish and Non-Scottish Residents"
    pattern = re.compile(r'(Male|Female|All)')
    
    geography_list = []
    updated_lookup_list = []
    
    for val in df[lookup_col]:
        if val.strip().startswith(special_case):
            geography_list.append("Scotland")
            updated_lookup_list.append(val.replace(special_case, "", 1).strip())
        else:
            match = pattern.search(val)
            if match:
                split_index = match.start()
                geography_list.append(val[:split_index].strip())
                updated_lookup_list.append(val[split_index:].strip())
            else:
                geography_list.append(val.strip())  # No keyword found
                updated_lookup_list.append("")
    
    df['Geography'] = geography_list
    df[lookup_col] = updated_lookup_list

    # Step 4: Extract Gender
    gender_list = []
    final_lookup_list = []
    
    for val in df[lookup_col]:
        if '-' in val:
            gender_part, rest = val.split('-', 1)
            gender_clean = gender_part.replace('Sexes', '').strip()
            gender_list.append(gender_clean)
            final_lookup_list.append(rest.strip())
        else:
            gender_list.append(val.replace('Sexes', '').strip())
            final_lookup_list.append("")
    
    df['Gender'] = gender_list
    df[lookup_col] = final_lookup_list

    # Step 5: Extract Age
    age_list = []
    last_lookup_list = []
    
    for val in df[lookup_col]:
        if val.lower().startswith('all ages'):
            age_list.append('All')
            rest = val[len('All ages'):].strip()
            last_lookup_list.append(rest)
        elif 'years' in val:
            age_part, rest = val.split('years', 1)
            age_list.append(age_part.strip())
            last_lookup_list.append(rest.strip())
        else:
            age_list.append(val.strip())
            last_lookup_list.append("")
    
    df['Age'] = age_list
    df[lookup_col] = last_lookup_list

    # Step 6: Rename lookup column to 'Condition'
    df.rename(columns={lookup_col: 'Condition'}, inplace=True)
    
    return df[['Year', 'Geography', 'Gender', 'Age', 'Condition', 
               'stays_Number', 'stays_Rate', 'los_stays_Number', 
               'los_stays_Rate', 'avlos_stays_Rate']]

In [22]:
hospital_activity_by_council_2019_20_df = pd.read_excel("../rawdata/table-6-main-diagnosis-by-council-area-2023-24.xlsx", 
                                                        sheet_name="Data1")

hospital_activity_by_council_2019_20_df_copy = process_hospital_activity_df(hospital_activity_by_council_2019_20_df)

In [23]:
hospital_activity_by_council_2020_21_df = pd.read_excel("../rawdata/table-6-main-diagnosis-by-council-area-2023-24.xlsx", 
                                                        sheet_name="Data2")

hospital_activity_by_council_2020_21_df_copy = process_hospital_activity_df(hospital_activity_by_council_2020_21_df)

In [24]:
hospital_activity_by_council_2021_22_df = pd.read_excel("../rawdata/table-6-main-diagnosis-by-council-area-2023-24.xlsx", 
                                                        sheet_name="Data3")

hospital_activity_by_council_2021_22_df_copy = process_hospital_activity_df(hospital_activity_by_council_2021_22_df)

In [25]:
hospital_activity_by_council_2022_23_df = pd.read_excel("../rawdata/table-6-main-diagnosis-by-council-area-2023-24.xlsx", 
                                                        sheet_name="Data4")

hospital_activity_by_council_2022_23_df_copy = process_hospital_activity_df(hospital_activity_by_council_2022_23_df)

In [26]:
hospital_activity_by_council_2023_24_df = pd.read_excel("../rawdata/table-6-main-diagnosis-by-council-area-2023-24.xlsx", 
                                                        sheet_name="Data5")

hospital_activity_by_council_2023_24_df_copy = process_hospital_activity_df(hospital_activity_by_council_2023_24_df)

In [27]:
def add_age_group_column(df):
    age_mapping = {
        'All': 'AllAges',
        '0-4': 'Ages0to4',
        '5-9': 'Ages5to14',
        '10-14': 'Ages5to14',
        '15-19': 'Ages15to24',
        '20-24': 'Ages15to24',
        '25-29': 'Ages25to44',
        '30-34': 'Ages25to44',
        '35-39': 'Ages25to44',
        '40-44': 'Ages25to44',
        '45-49': 'Ages45to64',
        '50-54': 'Ages45to64',
        '55-59': 'Ages45to64',
        '60-64': 'Ages45to64',
        '65-69': 'Ages65to74',
        '70-74': 'Ages65to74',
        '75-79': 'Ages75to84',
        '80-84': 'Ages75to84',
        '85+': 'Ages85plus',
        '85-89': 'Ages85plus',
        '90+': 'Ages85plus',
        # Special aggregates
        '65+': 'Special_65plus',
        '75+': 'Special_75plus',
        '<18': 'Special_Under18'
    }
    
    def map_age(age_val):
        return age_mapping.get(age_val, 'Unknown')
    
    df['Age_Group'] = df['Age'].apply(map_age)
    return df

In [28]:
hospital_activity_by_council_2019_20_df_copy = add_age_group_column(hospital_activity_by_council_2019_20_df_copy)
hospital_activity_by_council_2020_21_df_copy = add_age_group_column(hospital_activity_by_council_2020_21_df_copy)
hospital_activity_by_council_2021_22_df_copy = add_age_group_column(hospital_activity_by_council_2021_22_df_copy)
hospital_activity_by_council_2022_23_df_copy = add_age_group_column(hospital_activity_by_council_2022_23_df_copy)
hospital_activity_by_council_2023_24_df_copy = add_age_group_column(hospital_activity_by_council_2023_24_df_copy)

In [29]:
hospital_activity_by_council_2019_20_df_copy.to_csv('../data/hospital_activity_2019_20.csv', index=None)
hospital_activity_by_council_2020_21_df_copy.to_csv('../data/hospital_activity_2020_21.csv', index=None)
hospital_activity_by_council_2021_22_df_copy.to_csv('../data/hospital_activity_2021_22.csv', index=None)
hospital_activity_by_council_2022_23_df_copy.to_csv('../data/hospital_activity_2022_23.csv', index=None)
hospital_activity_by_council_2023_24_df_copy.to_csv('../data/hospital_activity_2023_24.csv', index=None)

<hr>